# Predictive Modeling

(PM.10) Predictive modeling is one of the main topics of data mining and can range from correlation to supervised segmentation

## Supervised Segmentation

In [1]:
#Global Functions
import init
from IPython.display import HTML

def scriptHTML():
    script = '''
        <script>
        
        function resizeIframe(ifrm) {       
            ifrm.style.height = ifrm.contentWindow.document.body.scrollHeight + 'px';
            // Setting the width here, or setting overflowX to "hidden" as above both 
            // work for this page. It may be that one turns out to be better.
            ifrm.style.width = ifrm.contentWindow.document.body.scrollWidth + 'px';
        }
        
        function code_toggle(identifier) {
        
            code_cell = $( "span:contains(" + identifier + "):last" ).parentsUntil('div.cell').last().parent().next().children().first();
            
            if ($(code_cell).is(':visible')) {
            code_cell.hide();
            }
            else
            {
            code_cell.show();
            }
            
            var ifrm = window.parent.document.getElementById('ipython_notebook_frame');   
            setTimeout(resizeIframe, 0, ifrm);
        
        }
        
        function hide_all() {
             $('div.input').hide();
        }

        $( document ).ready(hide_all);
         
        
        </script>
        '''
    
    return HTML(script)

def hide_code_message(identifier,message="Show/Hide Code"):
    form_start1 = ''' <form action=\"javascript:code_toggle('''
    identifier = "'" + identifier +  "'"
    form_start2 = ''')\"> <input type=\"submit\" value= '''
    form_end = '''></form>'''
    
    #print (form_start1 + identifier + form_start2 + '"' + message + '"' + form_end)
    return HTML(form_start1 + identifier + form_start2 + '"' + message + '"' + form_end)

scriptHTML()

In [2]:
hide_code_message("code-SS.10")

In [3]:
#Display movie history dataset
import init
import plotly
import plotly.tools as tls
from IPython.display import display
from IPython.display import Image
import cufflinks as cf
import pandas as pd
import numpy as np
import math
import plotly.plotly as py
import plotly.graph_objs as go


movie_history = pd.read_csv('https://goo.gl/xRfbvH')
display(movie_history)

,Title,Year,Genre,Length,Rating,Watched
0,Raiders of the Lost Ark,80s,Action,Average,Average,Yes
1,The Lord of the Rings 3,00s,Drama,Long,High,Yes
2,Fight Club,90s,Drama,Average,High,Yes
3,Braveheart,90s,Drama,Long,Average,Yes
4,Inception,10s,Action,Average,High,No
5,The Godfather,70s,Drama,Average,High,No
6,Aliens,80s,Horror,Average,Average,No
7,Once upon a time in America,80s,Crime,Long,Average,No


In [4]:
hide_code_message("code-MID.10","Show/Hide Code that calculates entropy")

(SS.10) The first concept within predictive modeling is supervised segmentation. Consider this example of a movie subscription service, like netflix, where there is a movie database that contains the history of whether a person watched a movie or not. Based on this history, we would like to predict which movie out of a potential list of unwatched movies, the person is likely to watch in the future.

(SS.20) So the target variable that we are interested in from the history is called *Watched*, which is a Yes or No value that represents whether the person watched a movie. Figuring out how we can find a formula, or in other words ***segment*** the history dataset to result in the value of *Watched = Yes*, is an example of supervised segmentation. 

(SS.30) Additionally, in this case we are talking about predicting a Yes or No value for the target variable *Watched*, which means we are doing a ***classification***. If we were predicting a numeric value such as the likelihood of watching a movie, instead of a Yes or No value, then we would be doing a ***regression***. In other words, a regression is to predict a numeric value for a target variable of interest.

(SS.40) The first part of supervised segmentation is to select the important informative attributes. The attributes we have in this dataset are *Title*,*Year*, *Genre*, *Length* and *Rating*. A single row of atrribute-values represent what is called a Feature vector. As an example, a feature vector here is: [ Title: The Godfather, Year : 70s, Genre: Drama, Length: Average, Rating: High ]. Each value in the feature vector is called a feature value. The attribute *Watched?* is the target variable.

(SS.50) So what is an informative attribute? An informative attribute is something that reduces uncertainity about the target variable. Let's assume for a second that we perfectly understand the person's preferences. The person prefers the drama genre over others, but the length of the movie doesn't matter to the person. In this case the *Genre* attribute is more informative, as in it reduces the uncertainity of predicting whether the person will watch a movie or not. However the *Length* attribute is not informative because, in this case, it doesn't contribute to reducing the uncertainty of whether or not the person will watch a movie.

## Models, induction and deduction

(MID.10) We need a systematic mechanism or *Model* in order to figure out which attributes are informative. The process of creating models from data is called **Induction** and the procedure that creates a model from the data is called a *learner* or an *induction algorithm*. Induction essentially refers to generalizing from a specific case to general rules. We use the process of induction to create models from training data, which in this case is the movie dataset and then use *deduction* to use the model to predict target values for other instances of feature vectors, which in this case is the list of unwatched movies.

(MID.20) Now let's try to create a model to figure out which attributes are more informative than others. The first concept is to quantify the randomness of the dataset for the target variable. To quantify the randomness of the dataset we use a concept called entropy.

(MID.30) Entropy is a measure of probability or relative percentage of each unique value occuring among all values in a dataset . Entropy is defined by the following mathematical formula:
$$entropy = - p_1 log (p_1) - p_2 log (p_2) - ...$$

In [5]:
# Calculates the entropy of the given data set for the target attribute.
def entropy(data, target_attr):
 
    val_freq = {}
    data_entropy = 0.0
 
    # Calculate the frequency of each of the values in the target attr
    for record in data:
        
        if (record[target_attr] in val_freq):
            val_freq[record[target_attr]] += 1.0
        else:
            val_freq[record[target_attr]]  = 1.0
 
    # Calculate the entropy of the data for the target attribute
    for freq in val_freq.values():
        data_entropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
 
    return round(data_entropy,2)

def calc_entropy(criteria,target, message):    
    print (message,entropy(movie_history[criteria].to_dict('records'),target))    
    return entropy(movie_history[criteria].to_dict('records'),target)

no_filter = calc_entropy(criteria=movie_history.index >= 0,target='Watched',message="Entropy of dataset:")

Entropy of dataset: 1.0


(MID.40) Where $p_1$, in this example, is the probability of the target variable *Watched = Yes* and $p_2$ is the probability of the target variable *Watched = No* in the result set. Simply looking at this formula tells us that if all records had values of *Watched = Yes*, then $p_1 = 1 and p_2 = 0$ and therefore $log (p_1) = 0 $ which makes the entropy equal 0. A segment with an entropy of 0 denotes a **pure** segment.

In [6]:
hide_code_message("code-MID.40","Show/Hide Code that generates the entropy plot")

In [7]:
#Plot entropy concept
data = [{'Watched': 'No'},
 {'Watched': 'No'},
 {'Watched': 'No'},
 {'Watched': 'No'},
 {'Watched': 'No'},
 {'Watched': 'No'},
 {'Watched': 'No'},
 {'Watched': 'No'}]


def target_values(d):
    vals = ""
    for i in range(0,len(data)):
        if i != 0:
            vals = vals + ","
        if i == len(data)/2:
            vals = vals + "<br>"
        vals = vals + d[i]['Watched']
    return vals

last_index = len(data) 
entropies = pd.DataFrame(columns=('X','Y','label'))
   
for i in range(0,last_index):        
    entropies.loc[i]= [i,entropy(data,"Watched"),target_values(data)]
    data[i]['Watched'] = 'Yes' 
entropies.loc[i+1]= [i+1,entropy(data,"Watched"),target_values(data)]

trace1 = go.Scatter(
    x=entropies.X,
    y=entropies.Y,
    mode='lines+markers+text',
    name='Lines, Markers and Text',
    text= entropies.label
)

display_data = [trace1]
layout = go.Layout(
    title = 
    'Entropy variation with the set of target values of <i>Watched</i> changing from all <b>No</b> to all <b>Yes</b> within the dataset',
    showlegend=False, font=dict(family='Source Sans Pro')
)
fig = go.Figure(data=display_data, layout=layout)

py.iplot(fig,filename='entropy_concept')

(MID.50) When you take the movie dataset of 8 rows and plot the entropy assuming all movies are unwatched, which means an entropy of 0, to assuming all movies are watched, which also means an entropy of 0, we find that the entropy is the highest at the midpoint where 50% of the movies are watched and 50% of movies are unwatched, which is the point of maximum randomness. Hence entropy helps us measure the general disorder of the segmentation which is an indicator that could be used to quantify how much a particular attribute increases or decreases the disorder of the segmentation produced. To increase the predictability of the segmentation, we want to indentify attributes that decrease the randomness of the segmentation.

In [21]:
movie_history[movie_history.Genre.str.contains('Drama')]

,Title,Year,Genre,Length,Rating,Watched
1,The Lord of the Rings 3,00s,Drama,Long,High,Yes
2,Fight Club,90s,Drama,Average,High,Yes
3,Braveheart,90s,Drama,Long,Average,Yes
5,The Godfather,70s,Drama,Average,High,No


(MID.60) In order to mathematically identify how much an attribute decreases randomness over the whole segmentation it creates, we use a concept called information gain that builds on the concept of entropy. 
To understand this better, lets filter this dataset by the attribute *Genre*. Looking at the movie history data, we see *Genre* has one of 4 values : Action, Drama, Horror, Crime. If we were to filter the history dataset by *Genre = Drama*, we would get 3 Watched movies and 1 unwatched movie and a resulting entropy of ~0.81. This entropy is less than the entropy of the whole dataset which was 1.0. Hence the information gain for a particular attrbute is derived as some function of change in entropy when the dataset is filtered or split on that attribute. When we plot the information gain for the attributes in the movie history dataset, we get the information gain values as 0.66, 0.34, 0.05, 0 for Year, Genre, Length and Rating respectively, showing that Year of the movie is the most informative attribute to predict whether a movie will be watched, followed by Genre and so on.

In [8]:
hide_code_message("code-MID.50")

In [9]:
genre_entropy = calc_entropy(criteria=movie_history.Genre.str.contains('Drama'),target='Watched',message="Entropy when Genre=Drama:")

Entropy when Genre=Drama: 0.81


In [22]:
hide_code_message("code-MID.60","Show/Hide Code to generate Information gain plot")

#Code to plot information gain
# Calculates the information gain (reduction in entropy) 
#that would result by splitting the data on the chosen attribute (attr).
def gain(data, attr, target_attr):
 
    val_freq = {}
    subset_entropy = 0.0
 
    # Calculate the frequency of each of the values in the target attribute
    for record in data:
        if (record[attr] in val_freq):
            val_freq[record[attr]] += 1.0
        else:
            val_freq[record[attr]]  = 1.0
 
    # Calculate the sum of the entropy for each subset of records weighted by 
    #their probability of occuring in the training set.
    for val in val_freq.keys():
        val_prob = val_freq[val] / sum(val_freq.values())
        data_subset = [record for record in data if record[attr] == val]
        subset_entropy += val_prob * entropy(data_subset, target_attr)
 
    # Subtract the entropy of the chosen attribute from the entropy of the whole
    #data set with respect to the target attribute (and return it)
    return round((entropy(data, target_attr) - subset_entropy),2)
 

def calc_information_gain(criteria,attr,target):        
    return gain(movie_history[criteria].to_dict('records'),attr, target)


target_variable = 'Watched'
gains = pd.DataFrame(columns=('Attribute','Gain'))
for i in range (0,len(movie_history.columns)):
    gains.loc[i]= [movie_history.columns.values[i],
                   calc_information_gain(movie_history.index >= 0,
                     movie_history.columns.values[i],target_variable)]

#Exclude target variable
gains = (gains[gains.Attribute.str.find(target_variable) == -1])  
gains = (gains[gains.Attribute.str.find('Title') == -1])    
trace1 = go.Bar(
    x=gains.Attribute,
    y=gains.Gain,
    text=gains.Gain    
)

display_data = [trace1]
layout = go.Layout(
    title = 
    'Information gain by attribute',
    showlegend=False, font=dict(family='Source Sans Pro')
)

fig = go.Figure(data=display_data, layout=layout)

py.iplot(fig,filename='information_gain_concept')

## Decision Trees

(DT.10) Finding the most informative attribute alone is likely insufficient to produce segments that are predictive of target values. There is an elegant mechanism to combine multiple informative attributes to produce supervised segmentation for the target variable. This mechanism is called a decision tree. Decision trees are often used as predictive models. When presented with an example for which we don't know a classification, we can predict it's classification by traversing the decision tree.

(DT.15) Decision trees are an important concept due to their ability to learn to classify individual records in a dataset. They are used in many areas, from typical marketing scenarios such as targeting to airplane autopilots and medical diagnoses. 

(DT.20) Using concepts we just learnt, let us build a decision tree model for our Movie subscription service to help us predict which movie out of a potential list of unwatched movies, the person is likely to watch in the future.

(DT.30) Creating a decision tree starts with choosing the most informative attribute in the dataset. This attribute becomes a **node** in the tree. For every value of this attribute, a child node is created. So, in our example, the most informative attribute is *Year*, and *Year* becomes our first node. *Year* has  possible values : 70s, 80s, 90s, 00s, 10s and a child node is created for each of these values.

<img style="align=middle" src="ipyimages/decision_tree1.png">

(DT.40) When we flter the dataset with the value at a node, we either get a pure dataset, which means all target values are the same at this node, or we get an impure dataset. If the dataset is pure, then this node is a leaf node. If the dataset is impure then the tree is built again at this node by picking the next best attribute. The process continues recursively till we reach all leaf nodes and this creates the decision tree. 

<img style="align=middle" src="ipyimages/decision_tree.png">

In [12]:
hide_code_message("code-DT.50","Show/Hide code that creates the decision tree")    

In [13]:
#
def choose_attribute(data_input, attributes, target_attr):
    """ Cycles through all the attributes and returns the attribute with the
    highest information gain """

    data = data_input[:]
    best_gain = 0.0
    best_attr = None

    for attr in attributes:
        if attr != target_attr:
            gain1 = gain(data, attr, target_attr)
            if gain1 > best_gain:
                best_gain = gain1
                best_attr = attr
    
    return best_attr

def majority_value(data, target_attr):
    
    dic = {}
    max_item = ""
    for record in data:
        dic[record[target_attr]] = dic.get(record[target_attr], 0) + 1
    counts = [(j,i) for i,j in dic.items()]
    count, max_item = max(counts)
    del dic
    return max_item

def get_subset( data_input, best, val):
    """ Returns a list of all the records in data with the value of attribute
    matching the given value """

    data = data_input[:]
    list = []

    if not data:
        return list
    else:
        for record in data:
            if record[best] == val:
                list.append(record)
        return list


def create_decision_tree(data_input, attributes, target_attr,attributes_only=False):
    """ Returns a new decision tree """
    #print ("--------------")
    #print ("\n")
    
    #display (data_input)
    #display (attributes)
    
    
    data    = data_input[:]
    vals    = [record[target_attr] for record in data]
    default = majority_value(data, target_attr)

    # If the dataset or attributes is empty, return the default value. Subtract
    # 1 to account for target attributes
    if not data or (len(attributes) - 1) <= 0:        
        return default
    # If all the records in dataset have the same values, return it
    elif vals.count(vals[0]) == len(vals):
        #print ("same values")
        return vals[0]
    else:
        # Choose the next best attribute
        best_attr = choose_attribute(data, attributes, target_attr)
        if (attributes_only):
            print (best_attr)
            
        # Create a new tree/node with the best attribute
        tree = {best_attr:{}}
        
        #print (best_attr)

        # Preprocess data, to generate a list containing the same data as data list
        # but without duplicate

        unique_data = []
        for record in data:
            if unique_data.count(record[best_attr]) <= 0:
                unique_data.append(record[best_attr])

        # Create a new decision tree for each of the values in the best
        # attribute field
        for val in unique_data:
            # Create a subtree for the current value under the best field
            subtree = create_decision_tree(
                    get_subset(data, best_attr, val),
                    [attr for attr in attributes if attr != best_attr],
                    target_attr, attributes_only
                    )

            # Add the new subtree to the empty dictionary
            tree[best_attr][val] = subtree

    return tree


attributes = movie_history.columns.tolist()
attributes.remove('Watched')
attributes.remove('Title')

def print_tree(tree,string):
    if type(tree) == dict:
        print (list(tree.keys())[0],"--->")        
        for item in list(tree.values())[0].keys():
            #print ('***begin***')
            print (string + "\t\t",item,':', end="")                          
            print_tree(list(tree.values())[0][item], string + "\t")
            #print ('***end***')
    else:
        #print ('else part')
        print (tree)        
   

In [14]:
movie_history2 = pd.read_csv('https://goo.gl/ZdIl1w')
tree=create_decision_tree(movie_history2.to_dict('records'),attributes,target_variable)        
print_tree(tree, "")

Year --->
		 80s :Genre --->
			 Horror :No
			 Action :Yes
			 Crime :No
		 00s :Yes
		 90s :Genre --->
			 Sci-Fi :Yes
			 Drama :Yes
			 Comedy :Yes
			 Action :No
			 Crime :Length --->
				 Average :Yes
				 Long :Yes
		 70s :No
		 10s :No
		 60s :No


(DT.60) Let us use our decision tree classifier built in python to classify the following three examples using our decision tree learner.

In [15]:
hide_code_message("code-DT.60")    

In [16]:
def classifier(tree, instance, default_class=None):
    '''Returns a classification label for instance, given a decision tree'''
    if not tree:  # if the node is empty, return the default class
        return default_class
    if not isinstance(tree, dict):  # if the node is a leaf, return its class label
        return tree
    attribute_index = list(tree.keys())[0]  # using list(dict.keys()) for Python 3 compatibility
    attribute_values = list(tree.values())[0]
    instance_attribute_value = instance[attribute_index]
    if instance_attribute_value not in attribute_values:  # this value was not in training data
        return default_class
    # recursively traverse the subtree (branch) associated with instance_attribute_value
    return classifier(attribute_values[instance_attribute_value], instance, default_class)     


def classify(examples):
    for i in range(0,len(examples)):        
        examples.Watched[i] = classifier(tree,examples.to_dict('records')[i])    
    return examples

#Examples to classify
examples = pd.read_csv('https://goo.gl/RFVTGk')
print ("Before Classification: ")
display(examples)
print ("After Classification: ")
display(classify(examples))
print("Decision Tree:")
print_tree(tree, "")

Before Classification: 


,Title,Year,Genre,Length,Rating,Watched
0,Reservoir Dogs,90s,Crime,Average,Average,?
1,Apocalypse Now,70s,Drama,Long,Average,?
2,Braveheart,90s,Drama,Long,Average,?


After Classification: 


,Title,Year,Genre,Length,Rating,Watched
0,Reservoir Dogs,90s,Crime,Average,Average,Yes
1,Apocalypse Now,70s,Drama,Long,Average,No
2,Braveheart,90s,Drama,Long,Average,Yes


Decision Tree:
Year --->
		 80s :Genre --->
			 Horror :No
			 Action :Yes
			 Crime :No
		 00s :Yes
		 90s :Genre --->
			 Sci-Fi :Yes
			 Drama :Yes
			 Comedy :Yes
			 Action :No
			 Crime :Length --->
				 Average :Yes
				 Long :Yes
		 70s :No
		 10s :No
		 60s :No


In [17]:
hide_code_message("code-DT.70")

(DT.70) When we have our learner classify 3 examples, we get 2 movies classified as Yes and one movie classified as No. Let's validate one result by traversing the decision tree. The first record is Reservoir Dogs, with *Year = 90s*. When we look at 90s in the decision tree, there is no pure result and hence the tree points us to look at *Genre*. The Genre for Reservoir Dogs is Crime, and when we look at Crime under Genre in the decision tree, again there is no pure result and the tree now points us to look at Length. The length of the movie Reservoir dogs is Average, and when we look at the decision tree we are at the leaf node, with a pure result of Yes, which matches the output from our classifier.

(DT.80) Tree-structured models work very well, though they may not be the most accurate model one can produce from a particular data set. This brings us to the end of this video where we talked about entropy, information gain and how they can ultimately be used within decision trees to create a learner out of a labeled dataset.